In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

#PDFs in BDT and sindec?
import os

# set env flags to catch BLAS used for scipy/numpy 
# to only use 1 cpu, n_cpus will be totally controlled by csky
if False:
    os.environ['MKL_NUM_THREADS'] = "1"
    os.environ['NUMEXPR_NUM_THREADS'] = "1"
    os.environ['OMP_NUM_THREADS'] = "1"
    os.environ['OPENBLAS_NUM_THREADS'] = "1"
    os.environ['VECLIB_MAXIMUM_THREADS'] = "1"

import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = 'w'
mpl.rcParams['savefig.facecolor'] = 'w'
import matplotlib.pyplot as plt
from matplotlib import colors, cm
import csky as cy
from csky import cext
import numpy as np
import astropy
#from icecube import astro
import histlite as hl
import healpy
import healpy as hp
import socket
import pickle
import copy
healpy.disable_warnings()
plt.rc('figure', facecolor = 'w')
plt.rc('figure', dpi=100)


#import seaborn as sns
#plt.rc('font', size=14)


## Define Settings

In [ ]:
selection_version = 'version-001-p01'

host_name = socket.gethostname()

if 'cobalt' in host_name:
    print('Working on Cobalts')
    plot_dir = '/data/user/mhuennefeld/data/analyses/DNNCascadeCodeReview/unblinding_checks/plots/unblinding/gp_flux_comparison'
    
else:
    raise ValueError('Unknown host:', host_name)

In [ ]:
for dir_path in [plot_dir]:
    if not os.path.exists(dir_path):
        print('Creating directory:', dir_path)
        os.makedirs(dir_path)

#### Define Colors

In [ ]:
colors = ['#0B3D53', '#FF7F0E',  '#5C9FC9']
colors

#### Load Templates

In [ ]:
kra5, _, _, ebins5 = np.load('/data/ana/analyses/NuSources/2021_DNNCascade_analyses/templates/KRA-gamma_5PeV_maps_energies.tuple.npy', allow_pickle = True, encoding='latin1')
kra50, _, _, ebins50 = np.load('/data/ana/analyses/NuSources/2021_DNNCascade_analyses/templates/KRA-gamma_maps_energies.tuple.npy', allow_pickle=True, encoding='latin1')
pi0 = np.load('/data/ana/analyses/NuSources/2021_DNNCascade_analyses/templates/Fermi-LAT_pi0_map.npy', allow_pickle=True, encoding='latin1')


In [ ]:

# the provided KRA-gamma files do not provide the uppermost bin edge.
# We will add it here (each edge is 10% higher than the last):
ebins5_ext = np.r_[ebins5, ebins5[-1] * 1.1]
ebins50_ext = np.r_[ebins50, ebins50[-1] * 1.1]

e_mids5 = ebins5_ext[:-1] + 0.5 * np.diff(ebins5_ext)
e_mids50 = ebins50_ext[:-1] + 0.5 * np.diff(ebins50_ext)


In [ ]:
import healpy as hp
a = hp.nside2pixarea(128)
print(a)

In [ ]:
dat = np.loadtxt('../paper/saved_sens/piecewise_fitresult.txt', skiprows = 13)
x = dat[:,0]
x_err_low = x - dat[:,1]
x_err_high = dat[:,2] - x

# from fit
tnorm = 1.0e-8 * (4*np.pi)

y = tnorm * dat[:,3]

y_max = tnorm * dat[:,-1]
y_min = tnorm * dat[:,-2]

y_err_high = y_max - y
y_err_low = y - y_min

indices_UL = np.array([5,8,9, 10, 11, 12])

# visualize upper limits with corresponding marker
y[indices_UL] = y[indices_UL]+y_err_high[indices_UL]

uplims = np.zeros(x.shape)
uplims[[indices_UL]]=True

# plot the points with non-zero best fit (i.e. not upper-limits)
x_new = np.delete(x, indices_UL)
x_err_high_new = np.delete(x_err_high, indices_UL)
x_err_low_new = np.delete(x_err_low, indices_UL)

y_new = np.delete(y, indices_UL)
y_err_high_new = np.delete(y_err_high, indices_UL)
y_err_low_new = np.delete(y_err_low, indices_UL)

In [ ]:
georg_flux = np.load('/data/user/gschwefer/misc/for_steve/fermi_pi0_gamma_ray_allsky_average_E2Phi_2.7_extrapol.npy', allow_pickle=True)
georg_energy = np.load('/data/user/gschwefer/misc/for_steve/fermi_pi0_gamma_ray_energies.npy', allow_pickle=True)
georg_flux_gamma = np.array(georg_flux)
georg_flux = 0.31* georg_flux # this is to convert gamma


In [ ]:
hans_result = np.loadtxt('../paper/saved_sens/Cascades_PRL_2010_11_12_13_14_15_pp_nominal_1sigma.txt', skiprows=1)

In [ ]:
hans_gammas = hans_result[:,0]
hans_norm = hans_result[:,1]

In [ ]:
hans_flux = {}
for i, gamma in enumerate(hans_gammas):
    Es = np.linspace(16000,2.6e6, 1000)
    hans_flux[gamma] =  Es**2*1e-18*hans_norm[i]*((Es/1e5)**(-1*hans_gammas[i]))
    
bestfithans = Es**2*1e-18*1.66*((Es/1e5)**-2.53)

In [ ]:
np.log10(16000)

In [ ]:
points = np.ndarray((len(hans_gammas), 1000))
xs = np.logspace(4.20411998,6.41497334,1000)

for i, gamma in enumerate(hans_gammas):
    points[i] = np.interp(np.log10(xs) ,np.log10(Es), hans_flux[gamma])
    

In [ ]:
print(points.shape)

mins = np.min(points, axis=0)
maxs = np.max(points, axis=0)

In [ ]:
#for gamma in (hans_gammas):
#    plt.plot(Es, hans_flux[gamma], c='grey')
#for i, gamma in enumerate(hans_gammas):
#    plt.plot(xs, points[i])
plt.fill_between(xs, mins, maxs, color='r', alpha=0.25)
plt.plot(Es, bestfithans, c='r', ls='--')
plt.loglog()
plt.ylim(5e-10,1e-6)
#plt.xlim(1e4,1e8)

In [ ]:
''' pi0flux = [1271.8527801661924, 0.020005181457912437,
1553.1845690893892, 0.01852439272816013,
1922.519128617942, 0.016478593938396482,
2381.49216131387, 0.014615905579883785,
2895.9554838722925, 0.012866607693166323,
3585.370348580028, 0.011101706303158538,
4375.590098581272, 0.009635108280540587,
5397.82609630125, 0.008285718315429003,
6645.846422152243, 0.007095591309854342,
8182.418973649131, 0.006067527085469145,
10074.25932040286, 0.005203618510847123,
12318.748238749182, 0.004495475813262984,
15166.940810483744, 0.0038723560749193814,
19149.109652203424, 0.0032775651321566347,
23576.536108990374, 0.0028150126763895286,
28772.831793568785, 0.0024502918550690176,
35494.8107159693, 0.002113304010356377,
43701.49326658316, 0.001825711674448281,
53717.853633984676, 0.0015706784002100286,
65793.2201885699, 0.0013546635088702511,
81005.13599200615, 0.0011600851314746696,
94194.95560159562, 0.001041064937747737
]
pi0_EGeV = np.array(pi0flux[::2]) /1e3
pi0_fluxGeV = np.array(pi0flux[1::2])  /1e3    
b1, b0, _r, _p_val, _stderr = scipy.stats.linregress(np.log10(pi0_EGeV[11:]), 
                                           np.log10(pi0_fluxGeV[11:])) 
x_pred = np.linspace(2,7, 100)
y_pred = b1 * x_pred + b0   '''

### Define energy ranges

GP paper uses data-derived TS-based ranges

In [ ]:
#erange_kra5 = [5000, 160000]
#erange_kra50 = [6500, 315000]
#erange_pi0 = [1800, 316000]

#ranges from ts
#erange_kra5 = [800, 100000]
#erange_kra50 = [1000, 146000]
#erange_pi0 = [800, 69000]

use_68_percent = False  # GP paper uses False, i.e. 90% range
if use_68_percent:
    erange_kra5 = [1891, 37145] #[1892, 37174] # TS-based 68%
    erange_kra50 = [2081, 50581] #[2084, 50604] # TS-based 68%
    erange_pi0 = [1295, 21155] # TS-based 68%

    erange_sens_kra5 = [10100, 81000] # sensitivity range 68%
    erange_sens_kra50 = [18200, 162000] # sensitivity range 68%
    erange_sens_pi0 = [3600, 99000] # sensitivity range 68%
else:
    erange_kra5 = [968, 99713] #[968, 99754] # TS-based 90%
    erange_kra50 = [1019, 145376] #[1019, 145377] # TS-based 90%
    erange_pi0 = [796, 68869] # TS-based 90%

    erange_sens_kra5 = [5200, 145000] # sensitivity range 90%
    erange_sens_kra50 = [6900, 357000] # sensitivity range 90%
    erange_sens_pi0 = [1500, 282000] # sensitivity range 90%
    

In [ ]:
mask5 = (ebins5 > erange_kra5[0]) & (ebins5 < erange_kra5[1])
mask5_sens = (ebins5 > erange_sens_kra5[0]) & (ebins5 < erange_sens_kra5[1])
mask50 = (ebins50 > erange_kra50[0]) & (ebins50 < erange_kra50[1])
mask50_sens = (ebins50 > erange_sens_kra50[0]) & (ebins50 < erange_sens_kra50[1])
#maskpi0 = (10**x_pred > 1500) & (10**x_pred < 315000)
maskpi0_georg = (georg_energy > np.log10(erange_pi0[0])) & (georg_energy < np.log10(erange_pi0[1]))
maskpi0_georg_gamma = (georg_energy > np.log10(erange_pi0[0]*2)) & (georg_energy < np.log10(erange_pi0[1]*2))
maskpi0_sens_georg = (georg_energy > np.log10(erange_sens_pi0[0])) & (georg_energy < np.log10(erange_sens_pi0[1]))
maskpi0_sens_georg_gamma = (georg_energy > np.log10(erange_sens_pi0[0]*2)) & (georg_energy < np.log10(erange_sens_pi0[1]*2))


## Define measured Flux and 1$\sigma$ uncertainties

In [ ]:
pi0modelnorm = 4.43e-12
pi0_range = [1.69e-11, 2.18e-11, 2.71e-11]
pi0_range = np.array(pi0_range) / pi0modelnorm
print(pi0_range)
print(pi0_range[0] - pi0_range[1])
print(pi0_range[2] - pi0_range[1])

In [ ]:
#pi0_range = [4.23, 5.45, 7]
kra5_range = [.398, .551,  0.726] # bugfix/TemplateFlux #[.650, .894,  1.18]
kra50_range = [.260, .372, 0.500] # bugfix/TemplateFlux #[.416, .592, 0.794]

In [ ]:
print(kra5_range[0] - kra5_range[1])
print(kra5_range[2] - kra5_range[1])

In [ ]:
print(kra50_range[0] - kra50_range[1])
print(kra50_range[2] - kra50_range[1])

### Helper Functions for flux plotting

In [ ]:
def powerlaw(energy, E2dNdE_at_100TeV, gamma=2.7):
    """Powerlaw flux

    Parameters
    ----------
    energy : array_like
        The energy in GeV at which to evaluate the powerlaw flux.
    E2dNdE_at_100TeV : array_like
        The powerlaw normalization in terms of E^2 dN/dE at 100 TeV
        in units of TeV^-1 cm^-2 s^-1.
    gamma : float, optional
        The spectral index.

    Returns
    -------
    array_like
        The powerlaw flux evaluated at the provided energies in terms
        of E^2 dN/dE in units of GeV^-1 cm^-2 s^-1.
    """
    e0 = 1e5  # 100 TeV

    # get normalization in dN/dE in units of GeV^-1 cm^-2 s^-1
    norm = E2dNdE_at_100TeV / (e0)**2 * 1e3
    dNdE = norm * (energy/e0)**(-gamma)
    return dNdE * energy**2

def plot_kra_flux(ax, template, bin_edges, model_norm=1., **kwargs):
    """Plot the KRA-gamma flux to a given axis.
    
    Parameters
    ----------
    ax : axis
        The matplotlib figure axis to which the flux will be plotted.
    template : array_like
        The model template binned over the sky and energy range in units
        of GeV^-1 * sr^-1 * s^-1 * cm^-2.
        Shape: [n_pix, n_bins]
    bin_edges : array_like
        The bin edges that define the energy bins of the model template.
        Shape: [n_bins + 1]
    model_norm : float, optional
        The model normalization to shift the nominal template flux by.
    **kwargs
        Keyword arguments passed on to ax.plot().
    """
    npix = len(template)
    nside = hp.npix2nside(npix)
    pixarea = hp.nside2pixarea(nside)
    
    # template provides all-flavor flux in
    # units of GeV^-1 * sr^-1 * s^-1 * cm^-2
    # integrate over the sky to obtain units of GeV^-1 * s^-1 * cm^-2
    # shape: [n_bins]
    flux = np.sum(template, axis=0) * pixarea
    
    # divide by three to obtain per-flavor neutrino flux
    flux /= 3.
    
    # scale by provided model normalization
    flux *= model_norm

    # compute bin centers
    # (Note that this does not account for energy spectrum within a bin)
    bin_mids = bin_edges[:-1] + 0.5*np.diff(bin_edges)
    
    # convert to E^2 * dN/dE in units of GeV * s^-1 * cm^-2
    E2_flux = bin_mids**2 * flux

    # plot flux
    ax.plot(bin_mids, E2_flux, **kwargs)

## Make plots

The plots here are just for testing purposes the GP flux plot (Figure 5) for the GP paper is created in a separate notebook.

In [ ]:
fig , ax = plt.subplots(figsize=(5.5,5.5))
plt.rc('font', size=10)
#plot_kra_flux(
#    ax=ax, 
#    template=kra5, 
#    bin_edges=ebins5_ext, 
#    model_norm=1., 
#    label='KRA$_\gamma^{5}$ Model 2',
#)
ax.plot(e_mids5, e_mids5**2 * (sum(kra5)) * hp.nside2pixarea(128) / 3 , ls=':', color=colors[0] ,
        lw=2, alpha=0.8, label='KRA$_\gamma^{5}$ Model')
ax.plot(e_mids50, e_mids50**2 * (sum(kra50)) * hp.nside2pixarea(128)/ 3, ls=':', color=colors[2],
        lw=2, alpha=0.8, label='KRA$_\gamma^{50}$ Model')
ax.plot(10**georg_energy, 4*np.pi*georg_flux, color=colors[1],
        lw=2, alpha=0.8, ls=':', label='$\pi^0$ Model')


ax.plot(np.nan, np.nan, label=' ', alpha=0)


#ax.plot(10 ** x_pred, 4*np.pi*10 ** y_pred, lw=3, linestyle = ":", 
#           label='$\pi^0$ Model')
'''(_, caps, _) = ax.errorbar(x[indices_UL], y[indices_UL], 
                           xerr=[x_err_low[indices_UL], x_err_high[indices_UL]], yerr=y[indices_UL]*0.5, fmt='o', 
                           c='dimgrey', linewidth=2.0, zorder=30, uplims=True, markersize=0)
(_, caps2, _) = ax.errorbar(x_new, y_new, xerr=[x_err_low_new, x_err_high_new], 
                            yerr=[y_err_low_new, y_err_high_new], fmt='o', c ='dimgrey', label="All-Sky Measured Flux", linewidth=2.0, zorder=31, markersize=0)
(_, caps4, _) = ax.errorbar(x_new, y_new, yerr=[y_err_low_new, y_err_high_new], 
                            fmt='o', c='dimgrey' , linewidth=2.0, zorder=31, markersize=0, capsize=0)

for cap in caps:
    cap.set_markeredgewidth(2)
'''
kra5_plot = e_mids5[mask5]**2 *(sum(kra5)[mask5]) * hp.nside2pixarea(128)/ 3

ax.plot(e_mids5[mask5], kra5_range[1] * kra5_plot , 
         c =colors[0], ls='-', zorder=10, lw=2, label='KRA$_\gamma^{5}$ Best-Fit Flux')
ax.fill_between(e_mids5[mask5], kra5_range[0] * kra5_plot , kra5_range[-1] * kra5_plot,
        color = colors[0], alpha=.25, zorder=10)

kra50_plot = e_mids50[mask50]**2 *(sum(kra50)[mask50]) * hp.nside2pixarea(128) / 3
ax.plot(e_mids50[mask50], kra50_range[1] * kra50_plot,
         c =colors[2], ls='-', zorder=5, lw=2, label='KRA$_\gamma^{50}$ Best-Fit Flux')
ax.fill_between(e_mids50[mask50],kra50_range[0] * kra50_plot, kra50_range[-1] * kra50_plot ,
        color = colors[2], zorder=5,  alpha=.25)

ax.plot(10**georg_energy[maskpi0_georg], pi0_range[1]*4*np.pi*georg_flux[maskpi0_georg], 
        lw=2, c=colors[1], ls='-', zorder=3,
        label='$\pi^0$ Best-Fit Flux')
ax.fill_between(10**georg_energy[maskpi0_georg], pi0_range[0]*4*np.pi*georg_flux[maskpi0_georg],
                pi0_range[-1]*4*np.pi*georg_flux[maskpi0_georg], zorder=3, color=colors[1], alpha=.25 )


#plt.plot(Es, 4*np.pi*bestfithans, c='dimgrey', lw='1', alpha=0.4, ls='--', label='M.G.Aartsen(2020)')
plt.fill_between(xs, 4*np.pi*mins, 4*np.pi*maxs, color='dimgrey', alpha=0.1,
                 edgecolor='k', label=r'M.G.$\,$Aartsen et al.(2020)')

plt.loglog()
#ax.set_ylim(1e-12,5e-9)
ax.set_ylim(5e-9,5e-6)
ax.set_xlim(500, 1e7)
plt.legend(ncol=2)
plt.rc('font', size=13)
ax.set_ylabel(r'E$_{\nu}^2$ $\frac{dN}{dE_\nu}$ [GeV s$^{-1}$ cm$^{-2}$]')
ax.set_xlabel(r'E$_\nu$ [GeV]')
plt.tight_layout()
fig.savefig('{}/gp_flux.png'.format(plot_dir))


In [ ]:
fig , ax = plt.subplots(figsize=(5.5,5.5))
plt.rc('font', size=10)
ax.plot(e_mids5, e_mids5**2 * (sum(kra5)) * hp.nside2pixarea(128) / (4*np.pi) , ls=':', color=colors[0] ,
        lw=2, alpha=0.8, label='KRA$_\gamma^{5}$ Model')
ax.plot(e_mids50, e_mids50**2 * (sum(kra50)) * hp.nside2pixarea(128)/ (4*np.pi), ls=':', color=colors[2],
        lw=2, alpha=0.8, label='KRA$_\gamma^{50}$ Model')
ax.plot(10**georg_energy, 3*georg_flux, color=colors[1],
        lw=2, alpha=0.8, ls=':', label='$\pi^0$ Model')

ax.plot(np.nan, np.nan, label=' ', alpha=0)

kra5_plot = e_mids5[mask5]**2 *(sum(kra5)[mask5]) * hp.nside2pixarea(128)/ (4*np.pi)

ax.plot(e_mids5[mask5], kra5_range[1] * kra5_plot , 
         c =colors[0], ls='-', zorder=10, lw=2, label='KRA$_\gamma^{5}$ Best-Fit Flux')
ax.fill_between(e_mids5[mask5], kra5_range[0] * kra5_plot , kra5_range[-1] * kra5_plot,
        color = colors[0], alpha=.25, zorder=10)

kra50_plot = e_mids50[mask50]**2 *(sum(kra50)[mask50]) * hp.nside2pixarea(128) / (4*np.pi)
ax.plot(e_mids50[mask50], kra50_range[1] * kra50_plot,
         c =colors[2], ls='-', zorder=5, lw=2, label='KRA$_\gamma^{50}$ Best-Fit Flux')
ax.fill_between(e_mids50[mask50],kra50_range[0] * kra50_plot, kra50_range[-1] * kra50_plot ,
        color = colors[2], zorder=5,  alpha=.25)

ax.plot(10**georg_energy[maskpi0_georg], 3*pi0_range[1]*georg_flux[maskpi0_georg], 
        lw=2, c=colors[1], ls='-', zorder=3,
        label='$\pi^0$ Best-Fit Flux')
ax.fill_between(10**georg_energy[maskpi0_georg], 3*pi0_range[0]*georg_flux[maskpi0_georg],
                3*pi0_range[-1]*georg_flux[maskpi0_georg], zorder=3, color=colors[1], alpha=.25 )


#plt.plot(Es, 4*np.pi*bestfithans, c='dimgrey', lw='1', alpha=0.4, ls='--', label='M.G.Aartsen(2020)')
plt.fill_between(xs, 3*mins,3* maxs, color='dimgrey', alpha=0.1,
                 edgecolor='k', label=r'M.G.$\,$Aartsen et al.(2020)')

plt.loglog()
#ax.set_ylim(1e-12,5e-9)
ax.set_ylim(1e-9,1.5e-6)
ax.set_xlim(500, 1e7)
plt.legend(ncol=2)
plt.rc('font', size=13)
ax.set_ylabel(r'E$_{6\nu}^2$ $\frac{dN}{dE_\nu}$ [GeV s$^{-1}$ cm$^{-2}$ sr$^{-1}$]')
ax.set_xlabel('E [GeV]')
plt.tight_layout()
fig.savefig('{}/gp_flux_sr.png'.format(plot_dir))


## Make Flux Comparison Plots for different Sky Regions

#### Helper Functions

In [ ]:
def plot_skymap(skymap, fig=None, ax=None, figsize=(9, 6), **kwargs):
    if ax is None:
        fig, ax = plt.subplots(
            subplot_kw=dict(projection='aitoff'), figsize=figsize)
    sp = cy.plotting.SkyPlotter()
    pc, cb = sp.plot_map(ax, skymap)
    return fig, ax


"""
def equatorial_to_galactic(m, rot=180):
    r = hp.Rotator(rot=rot, coord='CG')
    return r.rotate_map_pixel(m)

def equatorial_to_galactic_coords(theta, phi, rot=180):
    r = hp.Rotator(rot=rot, coord='CG')
    return r(theta, phi)

def ra_dec_to_lon_lat(ra, dec):
    r = hp.Rotator(rot=rot, coord='CG')
    return r(theta, phi)
"""

def template_to_galactic_coords(theta, phi):
    """Convert template coordinates (equatorial coordinates, 
    but defined in theta [0, pi] instead of dec [-pi/2, pi/2]) 
    to galactic coordinates, i.e. Galactic longitude l and
    Galactic latitude b.
    """

    theta = np.atleast_1d(theta)
    phi = np.atleast_1d(phi)

    r = hp.Rotator(rot=0, coord='CG')
    theta_gal, phi_gal = r(theta, phi)
    lat = theta_gal - np.pi/2.
    lon = phi_gal
    if isinstance(lon, float):
        if lon < 0:
            lon = lon + 2*np.pi
    else:
        lon[lon < 0] += 2*np.pi
    return lon, lat

def get_pixel_mask(lon_min_deg, lon_max_deg, lat_min_deg=-5, lat_max_deg=5, nside=128):
    
    # make sure the ranges are in correct order
    assert lon_min_deg < lon_max_deg, (lon_min_deg, lon_max_deg)
    assert lat_min_deg < lat_max_deg, (lat_min_deg, lat_max_deg)
    
    # both negative values
    lon_min_deg_other = None
    lon_max_deg_other = None
    if lon_min_deg < 0 and lon_max_deg <= 0:
        print('Both negative, old: {} {}'.format(lon_min_deg, lon_max_deg))
        lon_min_deg = 360 + lon_min_deg
        lon_max_deg = 360 + lon_max_deg
        print('Both negative, new: {} {}'.format(lon_min_deg, lon_max_deg))
    elif lon_min_deg < 0:
        print('Min negative, old: {} {}'.format(lon_min_deg, lon_max_deg))
        lon_min_deg_other = 360 + lon_min_deg
        lon_max_deg_other = 360
        lon_min_deg = 0
        print('Min negative, new: {} {}'.format(lon_min_deg, lon_max_deg))
        print('Min negative, other: {} {}'.format(lon_min_deg_other, lon_max_deg_other))
        

    lon_min = np.deg2rad(lon_min_deg)
    lon_max = np.deg2rad(lon_max_deg)
    lat_min = np.deg2rad(lat_min_deg)
    lat_max = np.deg2rad(lat_max_deg)

    npix = hp.nside2npix(nside)
    theta, phi = hp.pix2ang(nside, np.r_[:npix])

    lon, lat = template_to_galactic_coords(theta, phi)

    mask = np.logical_and(lon > lon_min, lon < lon_max)
    mask = np.logical_and(mask, lat > lat_min)
    mask = np.logical_and(mask, lat < lat_max)
    
    # now check other are if needed
    if lon_min_deg_other is not None:
        mask_other = get_pixel_mask(
            lon_min_deg=lon_min_deg_other, 
            lon_max_deg=lon_max_deg_other, 
            lat_min_deg=lat_min_deg, 
            lat_max_deg=lat_max_deg, 
            nside=nside,
        )
        mask = np.logical_or(mask, mask_other)
    return mask


In [ ]:
# ----------------------
# Run some sanity checks
# ----------------------
for cuts in [(25, 100), (50, 200), (-80, 80), (0, 80), (280, 360)]:
    pi0_masked = np.array(pi0)
    mask = get_pixel_mask(*cuts)
    print('mask', cuts, np.sum(mask))
    pi0_masked[~mask] = np.inf
    fig, ax = plot_skymap(pi0_masked)

max_ipix = np.argmax(pi0)
max_dir = np.rad2deg(hp.pix2ang(nside=hp.get_nside(pi0), ipix=max_ipix))
print('max', max_ipix, max_dir, np.rad2deg(template_to_galactic_coords(*np.deg2rad(max_dir))))
#plot_skymap(equatorial_to_galactic(pi0))
np.rad2deg(template_to_galactic_coords(theta=np.pi/2. - np.deg2rad(22), phi=np.deg2rad(83.63)))


#### Load Tibet Data

In [ ]:
import pandas as pd

df_tibet_points = pd.read_csv('data/plota_TibetData.csv')
df_tibet_points_b = pd.read_csv('data/plotb_TibetData.csv')
df_tibet_theory = pd.read_csv('data/plota_theory.csv')
df_lhasso_E2_TeV = pd.read_csv('data/plota_lhasso.csv')
df_FangMurase = pd.read_csv('data/plot__FangMurase.csv')
df_fermi_gamma = pd.read_csv('data/plot_fermi_gamma.csv')



def convert_E27_to_E20(energies, E27dNdE):
    E20dNdE = E27dNdE / (energies**0.7)
    return E20dNdE

# convert Tibet data points from E^-2.7 to E^-2
df_tibet_points_E2 = {}
df_tibet_points_E2b = {}
for key in ['TibetBestFit', 'TibetYUpper', 'TibetYLower', 'TibetXLower', 'TibetXUpper', 'ARGO-YBJBestFit', 'ARGO-YBJYLower', 'ARGO-YBJYUpper']:
    if key + '_x' in df_tibet_points:
        energies_gev = df_tibet_points[key + '_x'] * 1000.
        df_tibet_points_E2[key + '_x'] = energies_gev
        df_tibet_points_E2[key + '_y'] = convert_E27_to_E20(
            energies=energies_gev, E27dNdE=df_tibet_points[key + '_y'])

    if key + '_x' in df_tibet_points_b:
        energies_gev = df_tibet_points_b[key + '_x'] * 1000.
        df_tibet_points_E2b[key + '_x'] = energies_gev
        df_tibet_points_E2b[key + '_y'] = convert_E27_to_E20(
            energies=energies_gev, E27dNdE=df_tibet_points_b[key + '_y'])

df_tibet_points_E2 = pd.DataFrame(df_tibet_points_E2)
df_tibet_points_E2b = pd.DataFrame(df_tibet_points_E2b)

# convert Tibet theory points from E^-2.7 to E^-2
df_tibet_theory_E2 = {}
for key in ['SpaceIndependentCR', 'SpaceDependentCR']:
    energies_gev = df_tibet_theory[key + '_x'] * 1000.
    df_tibet_theory_E2[key + '_x'] = energies_gev
    df_tibet_theory_E2[key + '_y'] = convert_E27_to_E20(
        energies=energies_gev, E27dNdE=df_tibet_theory[key + '_y'])

df_tibet_theory_E2 = pd.DataFrame(df_tibet_theory_E2)

# convert LHAASO-KM2A points from TeV to GeV
df_lhasso_E2 = {}
for key in ['TibetBestFit', 'LHAASO-KM2A', 'LHAASO-KM2AYUpper', 'LHAASO-KM2AYLower']:
    df_lhasso_E2[key + '_x'] = df_lhasso_E2_TeV[key + '_x'] * 1000.
    df_lhasso_E2[key + '_y'] = df_lhasso_E2_TeV[key + '_y'] * 1000.
    
df_lhasso_E2 = pd.DataFrame(df_lhasso_E2)

df_tibet_points_E2


In [ ]:
df_fermi_gamma


#### Make Plots

In [ ]:
def plot_neutrino_flux(
            lon_min_deg, lon_max_deg, lat_deg, 
            show_gamma_equivalent_points=False,
            show_neutrino_equivalent_points=True,
            show_tibet=True,
            show_argo=True,
            show_lhasso=True,
            show_fang_murase=True,
            show_sens_range=True,
            show_model_flux=True,
            color_tibet='red',
            label_tibet=r'Tibet AS+MD [$E_\nu$ equivalent]',
            color_fang_murase='0.',
            alpha_fang_murase=0.25,
            label_fang_murase='K. Fang & K. Murase (2021)',
            xlabel=r'$E_\nu$ [GeV]',
            ylabel=r'E$_{\nu}^2$ $\frac{dN}{dE_\nu}$ [GeV s$^{-1}$ cm$^{-2}$ sr$^{-1}$]',
            label_diffuse=r'M.G.$\,$Aartsen et al.(2020)',
            fig=None, ax=None,
        ):
    # get healpix mask for region
    mask_area = get_pixel_mask(lon_min_deg=lon_min_deg, lon_max_deg=lon_max_deg, lat_min_deg=-lat_deg, lat_max_deg=lat_deg)

    # compute area of masked sky in steradian
    sky_area = hp.nside2pixarea(128) * np.sum(mask_area)

    # compute total flux per energy bin for KRA-5 model
    kra5_flux = sum(kra5[mask_area])
    kra50_flux = sum(kra50[mask_area])
    print('kra5 flux ratio:', np.sum(kra5_flux) / np.sum(kra5))
    print('kra50 flux ratio:', np.sum(kra50_flux) / np.sum(kra50))

    # compute fraction of pi0 flux that's located in this sky region
    pi0_flux_ratio = np.sum(pi0[mask_area]) / np.sum(pi0)
    print('Pi0 flux ratio:', pi0_flux_ratio)
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(9, 6))

    if (lon_min_deg, lon_max_deg, lat_deg) == (25, 100, 5):
        df_tibet = df_tibet_points_E2
        df_lhasso = df_lhasso_E2
        df_fm = None
    elif (lon_min_deg, lon_max_deg, lat_deg) == (50, 200, 5):
        df_tibet = df_tibet_points_E2b
        df_lhasso = None
        df_fm = None
    elif (lon_min_deg, lon_max_deg, lat_deg) == (0, 360, 5):
        df_tibet = None
        df_lhasso = None
        df_fm = df_FangMurase
    else:
        df_tibet = None
        df_lhasso = None
        df_fm = None

    if df_tibet is not None:
        x_conversion_factor = 0.5
        y_conversion_factor = 0.5 # 0.31
        
        #ax.scatter(df_tibet_points_E2['TibetBestFit_x'], df_tibet_points_E2['TibetBestFit_y'] * 0.31, label=r'Tibet AS+MD [$E_\gamma$ * 0.31]', color='red')
        xerr = np.array([
            (df_tibet['TibetBestFit_x'] - df_tibet['TibetXLower_x']),
            (df_tibet['TibetXUpper_x'] - df_tibet['TibetBestFit_x']),
        ])
        yerr = np.array([
            (df_tibet['TibetBestFit_y'] - df_tibet['TibetYLower_y']),
            (df_tibet['TibetYUpper_y'] - df_tibet['TibetBestFit_y']),
        ])
        if show_neutrino_equivalent_points and show_tibet:
            ax.errorbar(
                df_tibet['TibetBestFit_x'] * x_conversion_factor,
                df_tibet['TibetBestFit_y'] * y_conversion_factor,
                xerr=xerr * x_conversion_factor,
                yerr=yerr * y_conversion_factor,
                label=label_tibet, color=color_tibet, fmt='x',
            )
        if show_gamma_equivalent_points and show_tibet:
            ax.errorbar(
                df_tibet['TibetBestFit_x'],
                df_tibet['TibetBestFit_y'],
                xerr=xerr,
                yerr=yerr,
                label=r'Tibet AS+MD', color=color_tibet, fmt='.', alpha=0.3,
            )

        if 'ARGO-YBJBestFit_x' in df_tibet and show_argo:
            yerr = np.array([
                (df_tibet['ARGO-YBJBestFit_y'] - df_tibet['ARGO-YBJYLower_y']),
                (df_tibet['ARGO-YBJYUpper_y'] - df_tibet['ARGO-YBJBestFit_y']),
            ])
            
            if show_neutrino_equivalent_points:
                ax.errorbar(
                    df_tibet['ARGO-YBJBestFit_x'] * x_conversion_factor,
                    df_tibet['ARGO-YBJBestFit_y'] * y_conversion_factor,
                    yerr=yerr * y_conversion_factor,
                    label=r'ARGO-YBJ [$E_\nu$ equivalent]', color='blue', fmt='x',
                )
            
            if show_gamma_equivalent_points:
                ax.errorbar(
                    df_tibet['ARGO-YBJBestFit_x'],
                    df_tibet['ARGO-YBJBestFit_y'],
                    yerr=yerr,
                    label=r'ARGO-YBJ', color='blue', fmt='.',  alpha=0.3,
                )
        #ax.plot(df_tibet_theory_E2['SpaceIndependentCR_x'], df_tibet_theory_E2['SpaceIndependentCR_y'], label='Space Independent CR')
        #ax.plot(df_tibet_theory_E2['SpaceDependentCR_x'], df_tibet_theory_E2['SpaceDependentCR_y'], label='Space Independent CR')
    
    if df_lhasso is not None and show_lhasso:
        yerr = np.array([
            (df_lhasso['LHAASO-KM2A_y'] - df_lhasso['LHAASO-KM2AYLower_y']),
            (df_lhasso['LHAASO-KM2AYUpper_y'] - df_lhasso['LHAASO-KM2A_y']),
        ])
        
        if show_neutrino_equivalent_points:
            ax.errorbar(
                df_lhasso['LHAASO-KM2A_x'] * x_conversion_factor,
                df_lhasso['LHAASO-KM2A_y'] * y_conversion_factor,
                yerr=yerr * y_conversion_factor,
                label=r'(Preliminary) LHAASO-KM2A [$E_\nu$ equivalent]', color='purple', fmt='x',
            )
        if show_gamma_equivalent_points:
            ax.errorbar(
                df_lhasso['LHAASO-KM2A_x'],
                df_lhasso['LHAASO-KM2A_y'],
                yerr=yerr,
                label=r'(Preliminary) LHAASO-KM2A', color='purple', fmt='.',  alpha=0.3,
            )
    
    if df_fm is not None and show_fang_murase:
        all_to_single_flavor = 3.
        if False:
            ax.plot(
                df_FangMurase['KeFangKohtaMurase_5lat__lower_x'],
                df_FangMurase['KeFangKohtaMurase_5lat__lower_y'] / all_to_single_flavor,
                marker='.', color=color_fang_murase,
            )
            ax.plot(
                df_FangMurase['KeFangKohtaMurase_lat5__upper_x'],
                df_FangMurase['KeFangKohtaMurase_lat5__upper_y'] / all_to_single_flavor,
                marker='.', color=color_fang_murase,
            )
        ax.fill_between(
            df_FangMurase['KeFangKohtaMurase_5lat__lower_x'],
            df_FangMurase['KeFangKohtaMurase_5lat__lower_y'] / all_to_single_flavor,
            df_FangMurase['KeFangKohtaMurase_lat5__upper_y'] / all_to_single_flavor,
            color=color_fang_murase, zorder=5,  alpha=alpha_fang_murase, hatch='+', fc='1.0',
            label=label_fang_murase,
        )
        

    # plot measurements
    if True:
        kra5_plot = e_mids5**2 *(kra5_flux) * hp.nside2pixarea(128) / 3 / (sky_area)
        ax.plot(e_mids5, kra5_range[1] * kra5_plot ,
                 c =colors[1], ls='-', zorder=10, lw=2, label='KRA$_\gamma^{5}$ Best-Fit Flux')
        ax.fill_between(e_mids5[mask5], kra5_range[0] * kra5_plot[mask5] , kra5_range[-1] * kra5_plot[mask5],
                color = colors[1], alpha=.25, zorder=10)
        if show_sens_range:
            ax.fill_between(e_mids5[mask5_sens], kra5_range[0] * kra5_plot[mask5_sens] , kra5_range[-1] * kra5_plot[mask5_sens],
                color = colors[1], alpha=.25, zorder=10, hatch='///', fc='1.0',)

        kra50_plot = e_mids50**2 *(kra50_flux) * hp.nside2pixarea(128) / 3 / (sky_area)
        ax.plot(e_mids50, kra50_range[1] * kra50_plot,
                 c =colors[2], ls='-', zorder=5, lw=2, label='KRA$_\gamma^{50}$ Best-Fit Flux')
        ax.fill_between(e_mids50[mask50], kra50_range[0] * kra50_plot[mask50], kra50_range[-1] * kra50_plot[mask50],
                color = colors[2], zorder=5,  alpha=.25)
        if show_sens_range:
             ax.fill_between(e_mids50[mask50_sens],kra50_range[0] * kra50_plot[mask50_sens], kra50_range[-1] * kra50_plot[mask50_sens] ,
                color = colors[2], zorder=5,  alpha=.25, hatch='///', fc='1.0',)
    
        
        pi0_plot = georg_flux * (4*np.pi / sky_area) * pi0_flux_ratio
        ax.plot(10**georg_energy, pi0_range[1] * pi0_plot,
                lw=2, c=colors[0], ls='-', zorder=3,
                label='$\pi^0$ Best-Fit Flux')
        # TS-based range
        ax.fill_between(
            10**georg_energy[maskpi0_georg],
            pi0_range[0] * pi0_plot[maskpi0_georg],
            pi0_range[-1] * pi0_plot[maskpi0_georg],
            zorder=3, color=colors[0], alpha=.25
        )
        # Sensitivity-based range
        if show_sens_range:
            ax.fill_between(
                10**georg_energy[maskpi0_sens_georg],
                pi0_range[0] * pi0_plot[maskpi0_sens_georg],
                pi0_range[-1] * pi0_plot[maskpi0_sens_georg],
                zorder=3, color=colors[0], alpha=.25, hatch='///', fc='1.0',
            )


    # plot Diffuse
    if True:
        #ax.plot(Es, 4*np.pi*bestfithans, c='dimgrey', lw='1', alpha=0.4, ls='--', label='M.G.Aartsen(2020)')
        ax.fill_between(
            xs, mins, maxs, color='1.', alpha=0.1, edgecolor='0.0',
            label=label_diffuse,
            #label=r'M.G.$\,$Aartsen et al.(2020) [$E_\nu$ per-flavor]',
            hatch='///',
        )
    
    if show_model_flux:
        #ax.plot(10**georg_energy, georg_flux_gamma * (4*np.pi / sky_area) * pi0_flux_ratio, color=colors[1],
        #        lw=2, alpha=0.8, ls='-.', label=r'$\pi^0$ Model [$E_\gamma$]')
        ax.plot(10**georg_energy, georg_flux * (4*np.pi / sky_area) * pi0_flux_ratio, color=colors[1],
                lw=2, alpha=0.8, ls=':',
                #label=r'$\pi^0$ Model [$E_\nu$ per-flavor]',
                label=r'$\pi^0$ Model',
               )
        ax.plot(e_mids5, e_mids5**2 * (kra5_flux) * hp.nside2pixarea(128) / 3 / (sky_area), ls=':', color=colors[0] ,
                lw=2, alpha=0.8,
                #label=r'KRA$_\gamma^{5}$ Model [$E_\nu$ per-flavor]',
                label=r'KRA$_\gamma^{5}$ Model',
               )

    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_ylabel(ylabel)
    ax.set_xlabel(xlabel)
    ax.set_ylim(1e-9, 1.5e-6)
    ax.set_xlim(100, 1e7)
    #ax.set_ylim(7e-10, 3e-7) # limits from https://arxiv.org/pdf/2104.09491.pdf
    #ax.set_xlim(1e4, 2e6) # limits from https://arxiv.org/pdf/2104.09491.pdf
    ax.legend(ncol=2, fontsize=7)
    
    text = ('{}'.format(lon_min_deg) + r'$^\circ < l < $' + '{}'.format(lon_max_deg) +
        r'$^\circ$' + ', $|b| < $' + ' {}'.format(lat_deg) + r'$^\circ$')
    ax.text(
        0.02, 0.02, text,
        ha='left', va='bottom',
        color='0.7',
        transform=ax.transAxes
    )
    #ax.set_title(
    #    '{}'.format(lon_min_deg) + r'$^\circ < l < $' + '{}'.format(lon_max_deg) +
    #    r'$^\circ$, $|b| < $' + '{}'.format(lat_deg) + r'$^\circ$')
    
    return fig, ax


In [ ]:
show_gamma_equivalent_points = False
show_neutrino_equivalent_points = True

for lon_min_deg, lon_max_deg, lat_deg in [(25, 100, 5), (50, 200, 5), (-45, 45, 5), (0, 360, 90), (0, 360, 5)]:

    fig, ax = plot_neutrino_flux(
        lon_min_deg=lon_min_deg, 
        lon_max_deg=lon_max_deg, 
        lat_deg=lat_deg, 
        show_gamma_equivalent_points=show_gamma_equivalent_points,
        show_neutrino_equivalent_points=show_neutrino_equivalent_points,
    )
    fig.savefig('{}/gp_flux_comparison_l{:03d}_{:03d}_b{:02d}.png'.format(
        plot_dir, lon_min_deg, lon_max_deg, lat_deg))

#### GP Flux Comparison (Paper Plot)

In [ ]:
show_gamma_equivalent_points = False
show_neutrino_equivalent_points = True

fig, axes = plt.subplots(3, 1, sharex=True, figsize=(9, 8.5))

for ax, (lon_min_deg, lon_max_deg, lat_deg) in zip(axes, [(25, 100, 5), (50, 200, 5), (0, 360, 5)]):

    plot_neutrino_flux(
        lon_min_deg=lon_min_deg, 
        lon_max_deg=lon_max_deg, 
        lat_deg=lat_deg, 
        show_gamma_equivalent_points=show_gamma_equivalent_points,
        show_neutrino_equivalent_points=show_neutrino_equivalent_points,
        fig=fig, ax=ax,
        show_lhasso=False,
        show_argo=False,
        show_sens_range=False,
        show_model_flux=False,
        color_tibet='0.',
        #color_fang_murase='red',
        #alpha_fang_murase=1.,
        label_fang_murase=r'Prediction of diffuse Galactic $\nu$ flux (53)',
        label_diffuse=r'IceCube all-sky $\nu$ flux (24)',
        label_tibet=r'Tibet AS+MD [$E_\nu$ equivalent] (39)',
        xlabel=None,
    )
    ax.set_xlim(500, 3e6)
    ax.legend(ncol=2, fontsize=10)

# add panel labels
if True:
    for ax, label in zip(axes, ['A', 'B', 'C']):
        ax.text(
            .005, 0.98, label, 
            ha='left', va='top', color='0.', fontsize=18,
            transform=ax.transAxes,
        )

axes[-1].set_xlabel(r'$E_\nu$ [GeV]')
fig.tight_layout()

fig.savefig('{}/gp_neutrino_flux_comparison_combined.png'.format(
        plot_dir, lon_min_deg, lon_max_deg, lat_deg), dpi=300)
fig.savefig('{}/gp_neutrino_flux_comparison_combined.pdf'.format(
        plot_dir, lon_min_deg, lon_max_deg, lat_deg), dpi=300)

embargo_str = 'Under Embargo,\nNot For Proceedings'
if True:
    for ax in axes:
        ax.text(
            .6, 0.45, embargo_str, 
            ha='left', va='bottom', color='red', fontsize=18,
            transform=ax.transAxes,
        )
else:
    axes[-1].text(
        .6, 0.45, embargo_str, 
        ha='left', va='bottom', color='red', fontsize=18,
        transform=axes[-1].transAxes,
    )

fig.savefig('{}/gp_neutrino_flux_comparison_combined__embargo.png'.format(
        plot_dir, lon_min_deg, lon_max_deg, lat_deg), dpi=300)
fig.savefig('{}/gp_neutrino_flux_comparison_combined__embargo.pdf'.format(
        plot_dir, lon_min_deg, lon_max_deg, lat_deg), dpi=300)

#### Plot Gamma-Ray Fluxes

In [ ]:
def plot_gamma_ray_flux(
            lon_min_deg, lon_max_deg, lat_deg, 
            fig=None, ax=None,
        ):
    # get healpix mask for region
    mask_area = get_pixel_mask(lon_min_deg=lon_min_deg, lon_max_deg=lon_max_deg, lat_min_deg=-lat_deg, lat_max_deg=lat_deg)

    # compute area of masked sky in steradian
    sky_area = hp.nside2pixarea(128) * np.sum(mask_area)

    # compute fraction of pi0 flux that's located in this sky region
    pi0_flux_ratio = np.sum(pi0[mask_area]) / np.sum(pi0)
    print('Pi0 flux ratio:', pi0_flux_ratio)
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(9, 6))

    if (lon_min_deg, lon_max_deg, lat_deg) == (25, 100, 5):
        df_tibet = df_tibet_points_E2
        df_theory = df_tibet_theory_E2
        df_lhasso = df_lhasso_E2
        df_fg = None
    elif (lon_min_deg, lon_max_deg, lat_deg) == (50, 200, 5):
        df_tibet = df_tibet_points_E2b
        df_theory = None
        df_lhasso = None
        df_fg = None
    elif (lon_min_deg, lon_max_deg, lat_deg) == (-80, 80, 8):
        df_tibet = None
        df_theory = None
        df_lhasso = None
        df_fg = df_fermi_gamma
    else:
        df_tibet = None
        df_theory = None
        df_lhasso = None
        df_fg = None

    if df_tibet is not None:
        xerr = np.array([
            (df_tibet['TibetBestFit_x'] - df_tibet['TibetXLower_x']),
            (df_tibet['TibetXUpper_x'] - df_tibet['TibetBestFit_x']),
        ])
        yerr = np.array([
            (df_tibet['TibetBestFit_y'] - df_tibet['TibetYLower_y']),
            (df_tibet['TibetYUpper_y'] - df_tibet['TibetBestFit_y']),
        ])
        ax.errorbar(
            df_tibet['TibetBestFit_x'],
            df_tibet['TibetBestFit_y'],
            xerr=xerr,
            yerr=yerr,
            label=r'Tibet AS+MD', color='red', fmt='.',
        )
        
        if 'ARGO-YBJBestFit_x' in df_tibet:
            yerr = np.array([
                (df_tibet['ARGO-YBJBestFit_y'] - df_tibet['ARGO-YBJYLower_y']),
                (df_tibet['ARGO-YBJYUpper_y'] - df_tibet['ARGO-YBJBestFit_y']),
            ])
            
            ax.errorbar(
                df_tibet['ARGO-YBJBestFit_x'],
                df_tibet['ARGO-YBJBestFit_y'],
                yerr=yerr,
                label=r'ARGO-YBJ', color='blue', fmt='.',
            )
        if df_theory is not None:
            ax.plot(df_theory['SpaceIndependentCR_x'], df_theory['SpaceIndependentCR_y'], label='Space Independent CR', color='0.6', ls='-')
            ax.plot(df_theory['SpaceDependentCR_x'], df_theory['SpaceDependentCR_y'], label='Space Dependent CR', color='0.6', ls='--')

    if df_lhasso is not None:
        yerr = np.array([
            (df_lhasso['LHAASO-KM2A_y'] - df_lhasso['LHAASO-KM2AYLower_y']),
            (df_lhasso['LHAASO-KM2AYUpper_y'] - df_lhasso['LHAASO-KM2A_y']),
        ])
        
        ax.errorbar(
            df_lhasso['LHAASO-KM2A_x'],
            df_lhasso['LHAASO-KM2A_y'],
            yerr=yerr,
            label=r'(Preliminary) LHAASO-KM2A', color='black', fmt='.',
        )
    
    if df_fg is not None:
        ax.plot(
            df_fg['Fermi_fig17_pi0_x'],
            df_fg['Fermi_fig17_pi0_y'],
            label='Fermi-LAT $\pi^0$ (2012)',
        )
        ax.plot(
            df_fg['Fermi_fig17_total_x'],
            df_fg['Fermi_fig17_total_y'],
            label='Fermi-LAT total MC (2012)',
            color='0.1', ls='--',
        )
        ax.scatter(
            df_fg['Fermi_fig17_data_x'],
            df_fg['Fermi_fig17_data_y'],
            label='Fermi-LAT data (2012)',
            color='0.1', marker='.',
        )
        if False:
            
            # find closest index
            indices = np.searchsorted(10**georg_energy, df_fg['Fermi_fig17_total_x'])
            pi0_model_flux = georg_flux_gamma * (4*np.pi / sky_area) * pi0_flux_ratio
            ax.plot(
                df_fg['Fermi_fig17_total_x'],
                df_fg['Fermi_fig17_total_y'] + 1*pi0_model_flux[indices],
                label='Fermi-LAT total MC (2012) + $1\cdot \pi^0$',
                color='green', ls='-',
            )

    # plot measurements
    if True:
        ax.plot(10**georg_energy, pi0_range[1] * georg_flux_gamma * (4*np.pi / sky_area) * pi0_flux_ratio,
                lw=2, c=colors[1], ls='-', zorder=-3)
        ax.plot(10**georg_energy[maskpi0_georg], pi0_range[1] * georg_flux_gamma[maskpi0_georg] * (4*np.pi / sky_area) * pi0_flux_ratio,
                lw=2, c=colors[1], ls='-', zorder=3,
                label='$\pi^0$ Best-Fit Flux [$\gamma$-flux]')
        
        # TS-based range
        ax.fill_between(
            10**georg_energy[maskpi0_georg_gamma],
            pi0_range[0] * georg_flux_gamma[maskpi0_georg_gamma] * (4*np.pi / sky_area) * pi0_flux_ratio,
            pi0_range[-1] * georg_flux_gamma[maskpi0_georg_gamma] * (4*np.pi / sky_area) * pi0_flux_ratio,
            zorder=3, color=colors[1], alpha=.25,
            label=r'$\pi^0$ 68% TS Range $(\times 2)$',
        )
        
        # Sensitivity-based range
        ax.fill_between(
            10**georg_energy[maskpi0_sens_georg_gamma],
            pi0_range[0] * georg_flux_gamma[maskpi0_sens_georg_gamma] * (4*np.pi / sky_area) * pi0_flux_ratio,
            pi0_range[-1] * georg_flux_gamma[maskpi0_sens_georg_gamma] * (4*np.pi / sky_area) * pi0_flux_ratio,
            zorder=3, color=colors[1], alpha=.25, hatch='///', fc='1.0',
            label=r'$\pi^0$ 68% Sensitivity Range $(\times 2)$',
        )

    ax.plot(10**georg_energy, georg_flux_gamma * (4*np.pi / sky_area) * pi0_flux_ratio, color=colors[1],
            lw=2, alpha=0.8, ls='-.', label=r'$\pi^0$ Model [$\gamma$-flux]')

    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_ylabel(r'E$_{\gamma}^2$ $\frac{dN}{dE_\gamma}$ [GeV s$^{-1}$ cm$^{-2}$ sr$^{-1}$]')
    ax.set_xlabel(r'$E_\gamma$ [GeV]')
    ax.set_ylim(1e-9,1.5e-6)
    ax.set_xlim(300, 1e7)
    ax.set_ylim(1e-9, 8e-5)  # FERMI
    ax.set_xlim(1e-1, 1e7)  # FERMI
    ax.legend(ncol=2, fontsize=8)
    ax.set_title(
        '{}'.format(lon_min_deg) + r'$^\circ < l < $' + '{}'.format(lon_max_deg) +
        r'$^\circ$, $|b| < $' + '{}'.format(lat_deg) + r'$^\circ$')
    return fig, ax

In [ ]:
for lon_min_deg, lon_max_deg, lat_deg in [(25, 100, 5), (50, 200, 5), (-45, 45, 5), (0, 360, 90), (-80, 80, 8)]:
    
    fig, ax = plot_gamma_ray_flux(
        lon_min_deg=lon_min_deg, 
        lon_max_deg=lon_max_deg, 
        lat_deg=lat_deg, 
    )
    fig.savefig('{}/gp_gamma_flux_comparison_l{:03d}_{:03d}_b{:02d}.png'.format(
        plot_dir, lon_min_deg, lon_max_deg, lat_deg))

#### Scratch Area

In [ ]:
fig, ax = plot_skymap(pi0)

m1 = np.array(kra5[:, 20]) / np.sum(kra5[:, 20]) / pixarea
m2 = np.array(kra5[:, 150]) / np.sum(kra5[:, 150]) / pixarea
fig, ax = plot_skymap(np.log10(m1/m2))
#fig, ax = plot_skymap(np.log10(pi0))
for index in [0, 20, 100, 150]:
    pixarea = hp.nside2pixarea(hp.get_nside(kra5[:, index]))
    m = np.array(kra5[:, index]) / np.sum(kra5[:, index]) / pixarea
    #fig, ax = plot_skymap(m)
    #fig, ax = plot_skymap(np.log10(m))
    fig, ax = plot_skymap(np.log10(m / pi0))
    ax.set_title('E: {:3.3e}'.format(ebins5[index]))


In [ ]:
def powerlaw(x, gamma=-2.7):
    return x**gamma

e_primary = np.logspace(2, 7, 100)
e_gamma = e_primary * 0.1
e_nu = e_primary * 0.05
dNdE_primary = powerlaw(e_primary)

fig, ax = plt.subplots()
ax.plot(e_primary, e_primary**2*dNdE_primary, label='primary', color=colors[0])
ax.plot(e_nu, e_nu**2*dNdE_primary, label='nu', color=colors[2])
ax.plot(e_gamma, e_gamma**2*dNdE_primary, label='gamma', color=colors[1])
#ax.plot(e_gamma, e_gamma**2*dNdE_primary * 0.31, label='gamma', ls='--', color=colors[1])
ax.plot(e_gamma*.5, (e_gamma*.5)**2*dNdE_primary, label='gamma', ls='-.', color=colors[1])

ax.legend()
ax.set_xscale('log')
ax.set_yscale('log')
